# Задание по программированию: Линейная регрессия: прогноз оклада по описанию вакансии

In [1]:
import pandas as pd
from scipy.sparse import hstack
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


## 1. Загрузите данные об описаниях вакансий и соответствующих годовых зарплатах из файла salary-train.csv

In [2]:
train = pd.read_csv('data/salary-train.csv')
train.head(3)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000


## 2. Проведите предобработку:

Приведите тексты к нижнему регистру. Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова.

In [3]:
def text_transform(text):
    return (text
            .str.lower()
            .replace("[^a-zA-Z0-9]", " ", regex=True))

Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [4]:
vec = TfidfVectorizer(min_df=5)
X_train_text = vec.fit_transform(text_transform(train["FullDescription"]))

Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.

In [5]:
train["LocationNormalized"].fillna("nan", inplace=True)
train["ContractTime"].fillna("nan", inplace=True)

Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [6]:
enc = DictVectorizer()
X_train_cat = enc.fit_transform(train[["LocationNormalized", "ContractTime"]].to_dict("records"))

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [7]:
X_train = hstack([X_train_text, X_train_cat])

## 3. Обучите гребневую регрессию с параметрами alpha=1 и random_state=241.

Целевая переменная записана в столбце SalaryNormalized.

In [8]:
y_train = train["SalaryNormalized"]
model = Ridge(alpha=1, random_state=241)
model.fit(X_train, y_train)

Ridge(alpha=1, random_state=241)

## 4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv.

Значения полученных прогнозов являются ответом на задание. Укажите их через пробел.

In [9]:
test = pd.read_csv('data/salary-test-mini.csv')
test.head(3)

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [10]:
X_test_text = vec.transform(text_transform(test["FullDescription"]))
X_test_cat = enc.transform(test[["LocationNormalized", "ContractTime"]].to_dict("records"))
X_test = hstack([X_test_text, X_test_cat])

y_test = model.predict(X_test)
print(f"{y_test[0]:.2f} {y_test[1]:.2f}")

56567.06 37140.51


In [11]:
outFile_1 = open('01_Zadanie.txt', 'w', encoding='utf8')

print(f"{y_test[0]:.2f} {y_test[1]:.2f}", file=outFile_1)
outFile_1.close()